In [1]:
# We are Importing neccessary libraries

import zipfile
from scipy import stats
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
conn = sqlite3.connect("zippedData/im.db")

In [3]:
query = """
SELECT * 
 FROM movie_basics;
"""

# output query using pandas

movie_basics_df = pd.read_sql(query, conn)
movie_basics_df.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [4]:
query = """
SELECT *
 FROM movie_ratings;
"""

movie_rating_df = pd.read_sql(query, conn)
movie_rating_df.head()

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


In [10]:
# merge movie_basics_df & movie_rating_df

film_df = pd.merge(movie_basics_df, movie_rating_df, on='movie_id', how='inner')
film_df.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119


In [13]:
# load data from bom.movie_gross.csv.gz
# box_office_df : shows how movies were earning

box_office_df= pd.read_csv('zippedData/bom.movie_gross.csv.gz')
box_office_df.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [15]:
# Drop rows where either 'studio' or 'domestic_gross' has missing values
box_office_df.dropna(subset=["studio", "domestic_gross"], inplace=True)

# Check the shape of the DataFrame after dropping rows
print("Shape of box_office_df after dropping missing values in 'studio' or 'domestic_gross':")
print(box_office_df.shape)


Shape of box_office_df after dropping missing values in 'studio' or 'domestic_gross':
(3356, 5)


In [16]:
# Remove commas from 'foreign_gross' and convert it to a float
box_office_df["foreign_gross"] = box_office_df["foreign_gross"].replace(",", "", regex=True).astype(float)

# Fill missing values with the median of the 'foreign_gross' column
box_office_df["foreign_gross"].fillna(box_office_df["foreign_gross"].median(), inplace=True)

# Display the first few rows to check the results
print(box_office_df[['foreign_gross']].head())


   foreign_gross
0    652000000.0
1    691300000.0
2    664300000.0
3    535700000.0
4    513900000.0


In [17]:
# film Data Cleaning 

# Fill missing 'runtime_minutes' with the median value
median_runtime = film_df['runtime_minutes'].median()
film_df['runtime_minutes'].fillna(median_runtime, inplace=True)

# Fill missing 'genres' with 'Unknown'
film_df['genres'].fillna('Unknown', inplace=True)

# Optional: If you want to fill missing 'original_title' with the 'primary_title' instead
film_df['original_title'].fillna(movie_basics_df['primary_title'], inplace=True)

# --- Checking after cleaning ---
print("Missing Values After Cleaning:")
print(film_df.isnull().sum())


Missing Values After Cleaning:
movie_id           0
primary_title      0
original_title     0
start_year         0
runtime_minutes    0
genres             0
averagerating      0
numvotes           0
dtype: int64


In [18]:
median_runtime = film_df['runtime_minutes'].median()
movie_basics_df['runtime_minutes'].fillna(median_runtime, inplace=True)

# Fill missing 'genres' with 'Unknown'
movie_basics_df['genres'].fillna('Unknown', inplace=True)

# Optional: If you want to fill missing 'original_title' with the 'primary_title' instead
movie_basics_df['original_title'].fillna(movie_basics_df['primary_title'], inplace=True)

# --- Checking after cleaning ---
print("Missing Values After Cleaning:")
print(film_df.isnull().sum())

Missing Values After Cleaning:
movie_id           0
primary_title      0
original_title     0
start_year         0
runtime_minutes    0
genres             0
averagerating      0
numvotes           0
dtype: int64


In [19]:
# Drop any rows still missing 'runtime_minutes' or 'genres'
film_df.dropna(subset=['runtime_minutes', 'genres'], inplace=True)

# Final check
print("Missing Values After Final Cleaning:")
print(film_df.isnull().sum())


Missing Values After Final Cleaning:
movie_id           0
primary_title      0
original_title     0
start_year         0
runtime_minutes    0
genres             0
averagerating      0
numvotes           0
dtype: int64


In [20]:
# Standardize column names in film_df
film_df.columns = film_df.columns.str.strip().str.lower().str.replace(' ', '_')

# Standardize column names in box_office_df
box_office_df.columns = box_office_df.columns.str.strip().str.lower().str.replace(' ', '_')

# Display column names to confirm changes
print("Standardized Column Names in film_df:", film_df.columns.tolist())
print("Standardized Column Names in box_office_df:", box_office_df.columns.tolist())


Standardized Column Names in film_df: ['movie_id', 'primary_title', 'original_title', 'start_year', 'runtime_minutes', 'genres', 'averagerating', 'numvotes']
Standardized Column Names in box_office_df: ['title', 'studio', 'domestic_gross', 'foreign_gross', 'year']


In [21]:
# Correct inconsistent categorical values in film_df
for col in film_df.select_dtypes(include=['object']).columns:
    film_df[col] = film_df[col].str.lower().str.strip()

# Correct inconsistent categorical values in box_office_df
for col in box_office_df.select_dtypes(include=['object']).columns:
    box_office_df[col] = box_office_df[col].str.lower().str.strip()

# Displaying the first few rows to check the changes
print("First few rows of film_df after cleaning:")
print(film_df.head())

print("First few rows of box_office_df after cleaning:")
print(box_office_df.head())


First few rows of film_df after cleaning:
    movie_id                    primary_title              original_title  \
0  tt0063540                        sunghursh                   sunghursh   
1  tt0066787  one day before the rainy season             ashad ka ek din   
2  tt0069049       the other side of the wind  the other side of the wind   
3  tt0069204                  sabse bada sukh             sabse bada sukh   
4  tt0100275         the wandering soap opera       la telenovela errante   

   start_year  runtime_minutes                genres  averagerating  numvotes  
0        2013            175.0    action,crime,drama            7.0        77  
1        2019            114.0       biography,drama            7.2        43  
2        2018            122.0                 drama            6.9      4517  
3        2018             91.0          comedy,drama            6.1        13  
4        2017             80.0  comedy,drama,fantasy            6.5       119  
First few rows 

In [22]:
# Correct inconsistent casing and strip spaces in 'studio' and 'title' columns
box_office_df["studio"] = box_office_df["studio"].str.strip().str.lower()
box_office_df["title"] = box_office_df["title"].str.strip().str.lower()

# Displaying the first few rows to confirm the changes
print("First few rows of box_office_df after fixing casing:")
print(box_office_df[['studio', 'title']].head())


First few rows of box_office_df after fixing casing:
  studio                                        title
0     bv                                  toy story 3
1     bv                   alice in wonderland (2010)
2     wb  harry potter and the deathly hallows part 1
3     wb                                    inception
4   p/dw                          shrek forever after


In [23]:
# Differentiating the titles Bluebeard by adding the release year
box_office_df["title"] = box_office_df["title"] + " (" + box_office_df["year"].astype(str) + ")"
box_office_df["title"].value_counts()[box_office_df["title"].value_counts() > 1]

Series([], Name: title, dtype: int64)

In [24]:
# Check missing values in box_office_df and film_df
print("Missing Values in Box Office Data:\n", box_office_df.isnull().sum())
print("\nMissing Values in Merged Movie Data (film_df):\n", film_df.isnull().sum())

# Check duplicate rows in box_office_df and film_df
print("\nDuplicate Rows in Box Office Data:", box_office_df.duplicated().sum())
print("\nDuplicate Rows in Merged Movie Data (film_df):", film_df.duplicated().sum())

# Check data types in box_office_df and film_df
print("\nData Types in Box Office Data:\n", box_office_df.dtypes)
print("\nData Types in Merged Movie Data (film_df):\n", film_df.dtypes)


Missing Values in Box Office Data:
 title             0
studio            0
domestic_gross    0
foreign_gross     0
year              0
dtype: int64

Missing Values in Merged Movie Data (film_df):
 movie_id           0
primary_title      0
original_title     0
start_year         0
runtime_minutes    0
genres             0
averagerating      0
numvotes           0
dtype: int64

Duplicate Rows in Box Office Data: 0

Duplicate Rows in Merged Movie Data (film_df): 0

Data Types in Box Office Data:
 title              object
studio             object
domestic_gross    float64
foreign_gross     float64
year                int64
dtype: object

Data Types in Merged Movie Data (film_df):
 movie_id            object
primary_title       object
original_title      object
start_year           int64
runtime_minutes    float64
genres              object
averagerating      float64
numvotes             int64
dtype: object


In [26]:
film_df.to_csv("cleaned_film_dataset.csv", index=False)  # Save cleaned dataset for Tableau 
box_office_df.to_csv("cleaned_box_office_dataset.csv", index=False)
